In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


##Easy

In [20]:
merc_df = pd.read_csv('./cars/merc.csv')
merc_df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,SLK,2005,5200,Automatic,63000,Petrol,325,32.1,1.8
1,S Class,2017,34948,Automatic,27000,Hybrid,20,61.4,2.1
2,SL CLASS,2016,49948,Automatic,6200,Petrol,555,28.0,5.5
3,G Class,2016,61948,Automatic,16000,Petrol,325,30.4,4.0
4,G Class,2016,73948,Automatic,4000,Petrol,325,30.1,4.0
...,...,...,...,...,...,...,...,...,...
13114,C Class,2020,35999,Automatic,500,Diesel,145,55.4,2.0
13115,B Class,2020,24699,Automatic,2500,Diesel,145,55.4,2.0
13116,GLC Class,2019,30999,Automatic,11612,Diesel,145,41.5,2.1
13117,CLS Class,2019,37990,Automatic,2426,Diesel,145,45.6,2.0


In [21]:
num_cols = merc_df[['year', 'mileage', 'tax', 'mpg', 'engineSize']]
target = merc_df[['price']]
num_cols

,year,mileage,tax,mpg,engineSize
0,2005,63000,325,32.1,1.8
1,2017,27000,20,61.4,2.1
2,2016,6200,555,28.0,5.5
3,2016,16000,325,30.4,4.0
4,2016,4000,325,30.1,4.0
...,...,...,...,...,...
13114,2020,500,145,55.4,2.0
13115,2020,2500,145,55.4,2.0
13116,2019,11612,145,41.5,2.1
13117,2019,2426,145,45.6,2.0


In [22]:
x_train, x_test, y_train, y_test = train_test_split(num_cols, target, train_size=0.8, random_state=0)

In [23]:
tree = DecisionTreeRegressor()
tree.fit(x_train, y_train)
train_prediction = tree.predict(x_train)
test_prediction = tree.predict(x_test)

train_accuracy = r2_score(train_prediction, y_train)
test_accuracy = r2_score(test_prediction, y_test)

print('Точность на обучающей выборке: ', train_accuracy)
print('Точность на тестовой выборке: ', test_accuracy)
tree.feature_importances_

Точность на обучающей выборке:  0.9956476537300872
Точность на тестовой выборке:  0.8639807356654801


array([0.08998877, 0.33600831, 0.03570568, 0.19070747, 0.34758977])

Исходя из здравого смысла и того, как модель выбирает значимость каждого из признаков, вытекает вывод: цена больше всего зависит от пробега и объема двигателя. Причины тоже легко понять: чем больше объем, тем мощнее двигатель, то есть автомобиль более быстрый и престижный. И наоборот, чем больше пробег, тем дешевле машина, поскольку выше ее износ.

##Medium

In [24]:
import os

df = merc_df
for root, dirs, files in os.walk('./cars'):
    for file in files:
        if file not in ["unclean cclass.csv", "unclean focus.csv"] :
            buf = pd.read_csv('./cars' + os.sep + file)
            buf = buf.rename(columns = {'tax(£)':'tax'})  # change pounds into default currency
            #print(file, '\n', buf)
            df = pd.concat([df, buf], ignore_index = True)
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,SLK,2005,5200,Automatic,63000,Petrol,325.0,32.1,1.8
1,S Class,2017,34948,Automatic,27000,Hybrid,20.0,61.4,2.1
2,SL CLASS,2016,49948,Automatic,6200,Petrol,555.0,28.0,5.5
3,G Class,2016,61948,Automatic,16000,Petrol,325.0,30.4,4.0
4,G Class,2016,73948,Automatic,4000,Petrol,325.0,30.1,4.0
...,...,...,...,...,...,...,...,...,...
121654,Eos,2012,5990,Manual,74000,Diesel,125.0,58.9,2.0
121655,Fox,2008,1799,Manual,88102,Petrol,145.0,46.3,1.2
121656,Fox,2009,1590,Manual,70000,Petrol,200.0,42.0,1.4
121657,Fox,2006,1250,Manual,82704,Petrol,150.0,46.3,1.2


In [25]:
data = pd.get_dummies(df.drop('price', axis = 1), columns = ['model', 'transmission', 'fuelType'])
data = data.fillna(0)
target = df[['price']]
data

,year,mileage,tax,mpg,engineSize,model_ 1 Series,model_ 2 Series,model_ 3 Series,model_ 4 Series,model_ 5 Series,...,model_230,transmission_Automatic,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuelType_Diesel,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol
0,2005,63000,325.0,32.1,1.8,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
1,2017,27000,20.0,61.4,2.1,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
2,2016,6200,555.0,28.0,5.5,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
3,2016,16000,325.0,30.4,4.0,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
4,2016,4000,325.0,30.1,4.0,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121654,2012,74000,125.0,58.9,2.0,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False
121655,2008,88102,145.0,46.3,1.2,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
121656,2009,70000,200.0,42.0,1.4,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
121657,2006,82704,150.0,46.3,1.2,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [26]:
x_train, x_test, y_train, y_test = train_test_split(data, target, train_size=0.8, random_state=0)

In [27]:
tree = DecisionTreeRegressor()
tree.fit(x_train, y_train)
train_prediction = tree.predict(x_train)
test_prediction = tree.predict(x_test)

train_accuracy = r2_score(train_prediction, y_train)
test_accuracy = r2_score(test_prediction, y_test)

print('Точность на обучающей выборке: ', train_accuracy)
print('Точность на тестовой выборке: ', test_accuracy)

Точность на обучающей выборке:  0.9995619837364046
Точность на тестовой выборке:  0.9509177173768018


In [28]:
tree.feature_importances_.argmax()

201

In [29]:
list(data.keys())[201]

'transmission_Manual'

Сделаем перебор параметров

In [30]:
params = {
    'splitter' : ['best', 'random'],
    'max_depth': [20, 40, 60, 80]
}

search = GridSearchCV(DecisionTreeRegressor(), params, refit = True)
search.fit(x_train, y_train)
search.best_estimator_

DecisionTreeRegressor(max_depth=80, splitter='random')

In [31]:
best  = search.best_estimator_
train_prediction = best.predict(x_train)
test_prediction = best.predict(x_test)

train_accuracy = r2_score(train_prediction, y_train)
test_accuracy = r2_score(test_prediction, y_test)

print('Точность на обучающей выборке: ', train_accuracy)
print('Точность на тестовой выборке: ', test_accuracy)

Точность на обучающей выборке:  0.9995617644670527
Точность на тестовой выборке:  0.9609688001733079


Тяжелая артиллерия

In [32]:
x_train, x_test, y_train, y_test = train_test_split(df, target, train_size=0.8, random_state=0)

In [33]:
from catboost import CatBoostRegressor
forest = CatBoostRegressor(cat_features = ['model', 'transmission', 'fuelType'], max_depth = 9, iterations = 100) 
forest.fit(x_train, y_train)            # I heard that this is the best way to feed cat_cols into catboost
train_prediction = forest.predict(x_train)
test_prediction = forest.predict(x_test)

train_accuracy = r2_score(train_prediction, y_train)
test_accuracy = r2_score(test_prediction, y_test)

print('Точность на обучающей выборке: ', train_accuracy)
print('Точность на тестовой выборке: ', test_accuracy)

Learning rate set to 0.5
0:	learn: 5534.6322340	total: 259ms	remaining: 25.7s
1:	learn: 3182.2828037	total: 344ms	remaining: 16.8s
2:	learn: 1982.0104652	total: 433ms	remaining: 14s
3:	learn: 1463.3372902	total: 516ms	remaining: 12.4s
4:	learn: 1265.6291700	total: 602ms	remaining: 11.4s
5:	learn: 1149.0161743	total: 686ms	remaining: 10.8s
6:	learn: 1004.8419714	total: 778ms	remaining: 10.3s
7:	learn: 938.7073354	total: 861ms	remaining: 9.9s
8:	learn: 872.6824937	total: 953ms	remaining: 9.64s
9:	learn: 810.9413132	total: 1.04s	remaining: 9.34s
10:	learn: 775.9248125	total: 1.13s	remaining: 9.15s
11:	learn: 731.7638491	total: 1.22s	remaining: 8.93s
12:	learn: 680.5662418	total: 1.31s	remaining: 8.75s
13:	learn: 677.1249496	total: 1.39s	remaining: 8.56s
14:	learn: 634.4441696	total: 1.48s	remaining: 8.4s
15:	learn: 614.3736046	total: 1.57s	remaining: 8.23s
16:	learn: 614.2196535	total: 1.61s	remaining: 7.88s
17:	learn: 585.9904340	total: 1.7s	remaining: 7.74s
18:	learn: 573.5179646	total:

Как можно видеть, catboost справился лучше всего, но одиночное дерево тоже показало себя неплохо.
Выводы, которые можно сделать: нужно контролировать глубину ветвления. В случае с catboost я использовал max_depth = 9 (на основе личного опыта часто это оптимальный параметр - это как байка от Андрея Карпатого о том, что лучший learning_rate для Adam - это 3*10^(-4); но кроме шуток, в рамках ДЗ я вновь убедился в своем наблюдении, попробовав несколько других значений).
Любопытно, что в общем случае дерево выбрало самым главным параметром тип трансмиссии - но это тоже объяснимо: механическая коробка передач обычно стоит на относительно дешевых авто, для машин подороже существует большое число куда более сложных систем. 